### Setting up the environment

In [ ]:
%pip install nbformat
%pip install requests
%pip install configparser
%pip install ipywidgets
%pip install pandas
%pip install matplotlib

In [ ]:
%run ./helpers.ipynb
%run ./analytics.ipynb
%run ./pipeline.ipynb

In [ ]:
import ipywidgets as widgets
from IPython.display import display

### Main code block to run the pipeline

In [ ]:
# Initialize the logger
logger = get_logger()

# Here is where the pipeline runs & ingest data into Raw, Structured & Curated layers
try:
    logger.info('Starting the app!')

    # Load configs
    config = load_config('../../conf/config.ini')

    # Check if a Structured re=build is required before running the pipeline
    if (dict(config.items('arguments'))['rebuild'].lower() == 'true'):
        logger.info('Rebuild has started!')
        rebuild_structured_and_curated()

    # Run the ingestion pipeline
    logger.info('Starting ingestion pipeline!')
    run_pipeline(build_endpoint(config))

    logger.info('Pipeline has run successfully!')
except Exception as e:
    logger.exception(f'Something has gone wrong {e}')

### Analytics Section

In [ ]:
try:
    # Get the structured_rates dataset
    structured_path = '../../data/structured/structured_rates.json'
    if os.path.exists(structured_path):
        with open(structured_path, 'r') as file:
            data = json.load(file)
    
    rates_df = pd.DataFrame(data)
    
    # Get the summary_rates dataset
    curated_path = '../../data/curated/summary_rates.json'
    if os.path.exists(curated_path):
        with open(curated_path, 'r') as file:
            data = json.load(file)
    
    stats_df = pd.DataFrame(data)
except Exception as e:
    logger.exception(f'Something has gone wrong {e}')

In [ ]:
# Get a list of currency symbols from config file
symbols = ['All Currencies'] + dict(config.items('arguments'))['symbols'].split(',')

# Create interactive widget for selecting symbol
symbol_widget = widgets.Dropdown(
    options=symbols,
    description='Currency:'
)

# Define function to update plot based on selected symbol
def update_plot(symbol):
    df = summarize_exchange_rates(stats_df, symbol if symbol != 'All Currencies' else None)
    display(df)

    graph_exchange_rate_over_time(rates_df, symbol if symbol != 'All Currencies' else None)

In [ ]:
# Display the interactive widget
widgets.interact(update_plot, symbol=symbol_widget);

In [ ]:
graph_summary_exchange_by_currency(stats_df)